In [1]:
import pandas as pd

### Yelp Data

In [2]:
df = pd.read_csv("~/real_estate/data/v4/yelp_unsupervised_df.csv")

In [4]:
features = [
    "median_sqft_value","percent_decreasing","percent_increasing","zri_sqft_value","zhvi_condo",
    "zhvi_singlefam","mean_rating","bar_review_count","restaurant_review_count","coffee_review_count",
    "price_2_review_count","price_3_review_count","review_count","dependent_count_lag",
    "sum_adjusted_income_lag","sum_real_estate_taxes_lag","sum_mortgage_interest_paid_lag",
    "tax_return_count_lag","joint_return_count_lag","adult_count_lag","people_count_lag",
    "mean_adjusted_income_lag","mean_real_estate_taxes_hh_lag","mean_mortgage_interest_hh_lag",
    "mean_adjusted_income_with_dep_lag","90_day_treasury_bill_rate","mean_hourly_earnings_adj",
    "mean_fixed_mortgage_rate","total_nonfarm_payroll"
]

target = "zhvi_condo"

# Store data from previous, future months and years
feature_mo = {}
feature_yr = {}
zhvi_condo_12_months = []

for i in features:
    feature_mo[i+"_mo"] = []
    feature_yr[i+"_yr"] = []

In [7]:
for i in df.index:
    
    current_row = df.iloc[i]
    region_name = current_row["postal_code"]
    month = int(current_row["month"])
    year = int(current_row["year"])
    next_year = year + 1
    
    # Get column values of previous time preiods
    last_year = year - 1
    if month == 1:
        last_month_month = 12
        last_month_year = year - 1
    else:
        last_month_month = month - 1
        last_month_year = year
    
    last_month_row = df.loc[(df["postal_code"] == region_name) & 
                            (df["month"] == last_month_month) &
                            (df["year"] == last_month_year)]
    
    last_year_row = df.loc[(df["postal_code"] == region_name) &
                           (df["month"] == month) &
                           (df["year"] == last_year)]
    
    next_year_row = df.loc[(df["postal_code"] == region_name) &
                           (df["month"] == month) &
                           (df["year"] == next_year)]
    
    for i in features:
        
        feature_mo[i+"_mo"].append(last_month_row[i].values)
        feature_yr[i+"_yr"].append(last_year_row[i].values)
        
    next_year_target = next_year_row["zhvi_condo"]
    
    if len(next_year_target) == 0:
        next_year_target = None
    else:
        next_year_target = next_year_target.values[0]
    
    zhvi_condo_12_months.append(next_year_target)

In [8]:
# Add data to dataframe
for i in feature_mo.keys():
    new_list = [j[0] if len(j) > 0 else None for j in feature_mo[i]]
    feature_mo[i] = new_list

for i in feature_yr.keys():
    new_list = [j[0] if len(j) > 0 else None for j in feature_yr[i]]
    feature_yr[i] = new_list
    
for i in feature_mo.keys():
    df[i] = feature_mo[i]
    
for i in feature_yr.keys():
    df[i] = feature_yr[i]
    
df["zhvi_condo_12_months"] = zhvi_condo_12_months

In [10]:
# Filter
#df = df.dropna(subset=["total_nonfarm_payroll_yr","zhvi_condo_12_months"], how="any")

# Export
df.to_csv("~/real_estate/data/v4/yelp_supervised_df.csv", index=False)

In [11]:
print(df.shape)
df.head()

(25997, 92)


postal_code  median_sqft_value  percent_decreasing  percent_increasing  \
0         2109              582.0               16.79               75.38   
1         2109              583.0               12.27               81.85   
2         2109              585.0               12.35               82.61   
3         2109              582.0               12.12               82.23   
4         2109              578.0               17.09               74.17   

   zri_sqft_value  zhvi_condo  zhvi_singlefam  month  year  mean_rating  \
0             NaN    534000.0             NaN      1  2010          5.0   
1             NaN    536700.0             NaN      2  2010          5.0   
2             NaN    533200.0             NaN      3  2010          5.0   
3             NaN    526600.0             NaN      4  2010          5.0   
4             NaN    523400.0             NaN      5  2010          5.0   

           ...           mean_real_estate_taxes_hh_lag_yr  \
0          ...                                        NaN   
1          ...                                        NaN   
2          ...                                        NaN   
3          ...                                        NaN   
4          ...                                        NaN   

   mean_adjusted_income_lag_yr  joint_return_count_lag_yr  \
0                          NaN                        NaN   
1                          NaN                        NaN   
2                          NaN                        NaN   
3                          NaN                        NaN   
4                          NaN                        NaN   

   mean_adjusted_income_with_dep_lag_yr  restaurant_review_count_yr  \
0                                   NaN                         NaN   
1                                   NaN                         NaN   
2                                   NaN                         NaN   
3                                   NaN                         NaN   
4                                   NaN                         NaN   

   total_nonfarm_payroll_yr sum_adjusted_income_lag_yr  bar_review_count_yr  \
0                       NaN                        NaN                  NaN   
1                       NaN                        NaN                  NaN   
2                       NaN                        NaN                  NaN   
3                       NaN                        NaN                  NaN   
4                       NaN                        NaN                  NaN   

   zri_sqft_value_yr  zhvi_condo_12_months  
0                NaN              513900.0  
1                NaN              509700.0  
2                NaN              511300.0  
3                NaN              513800.0  
4                NaN              515500.0  

[5 rows x 92 columns]